In [ ]:
# importing the csv files 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error 
from sklearn.tree import DecisionTreeRegressor


df_amazon = pd.read_csv('amazon_data.csv')
df_coke = pd.read_csv('coke_data.csv')
df_jnj = pd.read_csv('j&j_data.csv')

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
df_amazon.head()
 

NameError: name 'df_amazon' is not defined

In [ ]:
# convert the "Date" column to a date and hour format with to_datetime() and creating separate columns
df_amazon['Date'] = pd.to_datetime(df_amazon['Date'])
 
df_amazon['date'] = df_amazon['Date'].dt.date
df_amazon['hour'] = df_amazon['Date'].dt.hour

df_amazon = df_amazon.drop(columns=['Date'])
 
df_amazon.head()

print(df_amazon)



In [ ]:
#here I'm flipping the data so we don't need to read data from bottom up 

df_amazon = df_amazon.iloc[::-1]
df_amazon = df_amazon.reset_index(drop=True)
#print to check

print(df_amazon)

In [ ]:

split_index = (len(df_amazon) - 1) // 2

# separate half to train later on and one for testing results
df_train_amazon = df_amazon.iloc[:split_index + 1].reset_index(drop=True)
df_test_amazon = df_amazon.iloc[split_index + 1:].reset_index(drop=True)

print(df_train_amazon)

print(df_test_amazon)

In [ ]:

plt.figure(figsize=(12, 6))
plt.plot(df_train_amazon.index, df_train_amazon['Open'], label='Open Price', linewidth=2)

plt.xlabel('Row #')
plt.ylabel('Open Price ($)')
plt.title('Open Prices by Row Number from 2024-9-18 to 2024-12-11')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:

# set x and y
X = df_train_amazon.index.to_numpy().reshape(-1, 1)
y = df_train_amazon['Open'].to_numpy()

# set polynomial regression degree
degree = 4
model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

model.fit(X, y)

# Predict the Open prices 
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)
print(r2)
# Plot it
plt.figure(figsize=(12, 6))
plt.plot(X, y, label='Actual Open Prices', linewidth=2)
plt.plot(X, y_pred, label=f'Polynomial Fit (degree {degree})', linewidth=2)

plt.xlabel('Row #')
plt.ylabel('Open Price ($)')
plt.title('Polynomial Regression Fit on trained data')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


degree 3 84.04

degree 4 86.05

degree 5 90.

In [ ]:
# now let see how it fits with the rest of the data
x_every = df_amazon.index.to_numpy().reshape(-1, 1)
y_every = df_amazon['Open'].to_numpy()

y_pred_half = model.predict(x_every)

plt.plot(x_every, y_every, label='Actual Open Prices', linewidth=2)
plt.plot(x_every, y_pred_half, label=f'Polynomial Fit (degree {degree})', linewidth=2)

plt.xlabel('Row #')
plt.ylabel('Open Price ($)')
plt.title('Polynomial Regression with everything now')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

This is a failed example as you can see from the top but what i can do is make a for loop where everytime it predicts the next hour and once next hour hits 

In [ ]:
start_size = (len(df_amazon) - 1) // 2

predictions = []    
actuals = [] 

for i in range(start_size, len(df_amazon)):
    train_data = df_amazon.iloc[i - start_size:i]
    X_train = np.arange(start_size).reshape(-1, 1)
    y_train = train_data['Open'].to_numpy()  
    
    # linear
    #model = LinearRegression()
    #model.fit(X_train, y_train)

    degree = 3  
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(X_train, y_train)
    
    # Predict the next hour
    next_index = np.array([[start_size]])  # the next index after the training window
    pred = model.predict(next_index)
    
    # Save the prediction and the actual value
    predictions.append(pred[0])
    actuals.append(df_amazon.iloc[i]['Open'])

results = pd.DataFrame({
    'Actual_Open': actuals,
    'Predicted_Open': predictions
}, index=df_amazon.index[start_size:])

plt.figure(figsize=(12, 6))
plt.plot(results.index, results['Actual_Open'], label='Actual Open Price', linewidth=2)
plt.plot(results.index, results['Predicted_Open'], label='Predicted Open Price', linestyle='--', linewidth=2)
plt.xlabel('Row in Hours')
plt.ylabel('Open Price ($)')
plt.title('Hourly Stock Price Prediction with Looping Regression (Degree 3)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Metrics for the degree 3 model
mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R² Score:", r2)




In [ ]:
# Calculate residuals
residuals = np.array(actuals) - np.array(predictions)

plt.figure(figsize=(12, 6))
plt.hist(residuals, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Residual (Actual - Predicted)')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals for Degree 3 Model')
plt.grid(True)
plt.tight_layout()
plt.show()


Overall, the histogram suggests the model frequently overpredicts by a few dollars (seeing more on negative side in the distribution) and sometimes makes large errors in both directions. This we can investigate later on to further learn what is causing the changes in price prediction. 

In [ ]:
start_size = (len(df_amazon) - 1) // 2

predictions = []    
actuals = [] 

for i in range(start_size, len(df_amazon)):
    train_data = df_amazon.iloc[i - start_size:i]
    X_train = np.arange(start_size).reshape(-1, 1)
    y_train = train_data['Open'].to_numpy()  
    
    # degree-4 polynomial regression model
    degree = 4  
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(X_train, y_train)
    
    # Predict the next hour
    next_index = np.array([[start_size]])  # the next index after the training window
    pred = model.predict(next_index)
    
    # Save the prediction and the actual value
    predictions.append(pred[0])
    actuals.append(df_amazon.iloc[i]['Open'])

results = pd.DataFrame({
    'Actual_Open': actuals,
    'Predicted_Open': predictions
}, index=df_amazon.index[start_size:])

plt.figure(figsize=(12, 6))
plt.plot(results.index, results['Actual_Open'], label='Actual Open Price', linewidth=2)
plt.plot(results.index, results['Predicted_Open'], label='Predicted Open Price', linestyle='--', linewidth=2)
plt.xlabel('Row in Hours')
plt.ylabel('Open Price ($)')
plt.title('Hourly Stock Price Prediction with Looping Regression (Degree 4)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Metrics for the degree 4 model
mse = mean_squared_error(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R² Score:", r2)



As we can see from the visualization and performance metrics, degree 3 model performs better. The degree 3 model has lower MSE and MAE values indicating that the average squared difference between the predicted and actual values is smaller and  average absolute error is reduced. Degree 3 model also has a higher R-squared score of 0.651 compared to degree 4's 0.577 implying it explains a greater portion of the variance in the data. 

In [ ]:
# Now we will create the decision tree model with the trainining dataset (TODO add more intro above, connect it with previous step's)
start_size = (len(df_amazon) - 1) // 2

predictions = []
actuals = []

for i in range(start_size, len(df_amazon)): 
    train_data = df_amazon.iloc[i - start_size : i]
    
    X_train = np.arange(start_size).reshape(-1, 1)
    y_train = train_data['Open'].to_numpy()

    # Initialize and train a decision tree model 
    model = DecisionTreeRegressor(max_depth=5, random_state=42)
    model.fit(X_train, y_train)

    # Predict the next point (index = start_size)
    next_index = np.array([[start_size]])  
    pred = model.predict(next_index)

    predictions.append(pred[0])
    actuals.append(df_amazon.iloc[i]['Open'])

# Compare predictions vs. actual price
results = pd.DataFrame({
    'Actual_Open': actuals,
    'Predicted_Open': predictions
}, index=df_amazon.index[start_size:])

# Calculate error metrics
mse = mean_squared_error(results['Actual_Open'], results['Predicted_Open'])
mae = mean_absolute_error(results['Actual_Open'], results['Predicted_Open'])
r2 = r2_score(results['Actual_Open'], results['Predicted_Open'])
print("Decision Tree (max_depth=5) Performance:")
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R² Score:", r2)

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(results.index, results['Actual_Open'], label='Actual Open Price', linewidth=2)
plt.plot(results.index, results['Predicted_Open'], label='Predicted Open Price', linestyle='--', linewidth=2)
plt.xlabel('Row in Hours')
plt.ylabel('Open Price ($)')
plt.title('Hourly Stock Price Prediction with Decision Tree Regression')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# calculating residuals for the decision tree model
residuals = results['Actual_Open'] - results['Predicted_Open']

# histogram of residuals
plt.figure(figsize=(12, 6))
plt.hist(residuals, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Residual (Actual - Predicted)')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals for Decision Tree Model')
plt.grid(True)
plt.tight_layout()
plt.show()

TODO (esther): write about the results 